fix me
- basic website with setup info. new features etc. 
- inflections can you colour 1st and 2nd / 3rd fourth columns differetnly?
- inflections spaces between verbs
- inflections 1st line of text {hadowrd} is "{pattern}" like {like}
- inflections active sg active pl reflexive sg reflexive pl
- split inflction list generator into 1. tables and synonms 2. generate all words
- inflections </table>
- root families split  1. for anki 2. for dpd 3. etc.
- declension or conjugation label, declensions pos = ["masc", "fem", "nt" etc]
- family2 outside range border range

- open to sharing once all feedback completed

- eccosystem
- - goldendict
- - google sheets feedback
- - google doc install instructions
- - google drive download update
  
- help
- abbreviations
- bibliography

- in comps families,  exlcude word itself

setup

In [6]:
import pandas as pd
import re
import json
import pickle
import os
from datetime import date

today = date.today()

df = pd.read_csv("/home/bhikkhu/Bodhirasa/Dropbox/dpd/csvs/dpd-full.csv", sep = "\t", dtype=str)
df = df.fillna("") 
df_length = df.shape[0]

generate html & json

In [7]:
gd_js = open ("/home/bhikkhu/Bodhirasa/Dropbox/dpd/exporter/output/gd.json", "w", encoding ='utf8')
ajdict = open ("/home/bhikkhu/Bodhirasa/Dropbox/dpd/exporter/output/ajdict.txt", "w", encoding ='utf8')
error_log = open("errors.txt", "w")

for row in range(df_length): #df_length

# define variables
	pali = df.loc[row, "Pāli1"]
	if row % 5000 == 0:
		print(row, pali)
	pali_ = "_" + re.sub(" ", "_", pali) 
	pali2 = df.loc[row, "Pāli2"]
	pali_clean = re.sub(" \d*$", "", pali)
	pos = df.loc[row, "POS"]
	grammar = df.loc[row, "Grammar"]
	neg = df.loc[row, "Neg"]
	verb = df.loc[row, "Verb"]
	trans = df.loc[row, "Trans"]
	case = df.loc[row, "Case"]
	meaning = df.loc[row, "Meaning IN CONTEXT"]
	lit = df.loc[row, "Literal Meaning"]
	buddhadatta = df.loc[row, "Buddhadatta"]
	non_ia = df.loc[row, "Non IA"]
	sk = df.loc[row, "Sanskrit"]
	sk_root = df.loc[row, "Sk Root"]
	sk_root_mn = df.loc[row, "Sk Root Mn"]
	sk_root_cl = df.loc[row, "Cl"]
	root = df.loc[row, "Pāli Root"]
	root_in_comps = df.loc[row, "Root In Comps"]
	root_verb = df.loc[row, "V"]
	root_grp = df.loc[row, "Grp"]
	root_sign = df.loc[row, "Sgn"]
	root_meaning = df.loc[row, "Root Meaning"]
	base = df.loc[row, "Base"]
	family = df.loc[row, "Family"]
	family2 = df.loc[row, "Family2"]
	construction =  df.loc[row, "Construction"]
	derivative = df.loc[row, "Derivative"]
	suffix = df.loc[row, "Suffix"]
	pc = df.loc[row, "Phonetic Changes"]
	comp = df.loc[row, "Compound"]
	comp_constr = df.loc[row, "Compound Construction"]
	source1 = df.loc[row, "Source1"]
	sutta1 = df.loc[row, "Sutta1"]
	eg1 = df.loc[row, "Example1"]
	source2 = df.loc[row, "Source 2"]
	sutta2 = df.loc[row, "Sutta2"]
	eg2 = df.loc[row, "Example 2"]
	ant = df.loc[row, "Antonyms"]
	syn = df.loc[row, "Synonyms – different word"]
	var = df.loc[row, "Variant – same constr or diff reading"]
	comm = df.loc[row, "Commentary"]
	comm = re.sub("(.+)\.$", "\\1", comm)
	notes = df.loc[row, "Notes"]
	cognate = df.loc[row, "Cognate"]
	category = df.loc[row, "Category"]
	metadata = df.loc[row, "Metadata"]
	link = df.loc[row, "Link"]

	colour1 = "#66FF99"
	colour2 = "#00A4CC"

	indeclinables = ["abbrev", "abs", "ger", "ind", "inf", "prefix"]
	conjugations = ["aor", "cond", "fut", "imp", "imperf", "opt", "perf", "pr"]
	declensions = ["adj", "card", "cs", "fem", "letter", "masc", "nt", "ordin", "pp", "pron", "prp", "ptp", "root", "suffix", "ve"]
	

# htm head & stylel

	text_long = open(f"output/text_long/{pali}.txt", "w")
	html_file = open(f"output/html/{pali}.html", "w")
		
	html_file.write(f"""<html><head><meta charset="utf-8"></head>""")
	html_file.write("""<style>""")
	html_file.write("""* {font-family: "Verajja Serif", "DejaVu Sans";}""")
	html_file.write("""p {vertical-align: center; line-height: 170%; margin: 5px 10px 5px 10px;}""")
	# html_file.write("""p:hover { background-colour: #00A4CC; }""")

	html_file.write("""table.table1 td, table.table1 th { text-align: left; vertical-align: top; line-height: 150%; margin: 5px 10px 5px 10px; padding: 2px 5px 2px 5px ;  background-color:#00A4CC25; }""")
	html_file.write("""table.table1 th { color:#00A4CC; font-weight:normal; background-color:#00A4CC10;  }""")
	html_file.write("""table.table1 td, table.table1 th {background-color:#00A4CC10; border: 2px solid white; transition: ease .2s;}""")
	html_file.write("""table.table1 tr:hover { background-color:#00A4CC20;}""")

	html_file.write("""table.table2 td, table.table2 th { text-align: center; vertical-align: center; line-height: 150%; padding: 5px 5px 5px 5px; background-color:#00A4CC25; } """)
	html_file.write(""" table.table2 {overflow-clip-margin: 0px; }""")	
	# html_file.write("""table.table2 td { border: 1px solid white; transition: ease .2s;} }""")
	html_file.write("""table.table2 th { color:#00A4CC; font-weight:normal; background-color:#00A4CC10; }""")
	html_file.write("""table.table2 td:hover, table.table2 th:hover{background-color:#00A4CC50;}""")

	html_file.write("""table.table3, table.table3 td, table.table3 th {text-align: left; vertical-align: top; line-height: 150%; margin: 0px 0px 0px 0px; padding: 2px 5px 2px 5px; overflow-clip-margin: 0px;} """)
	html_file.write("""table.table3 td,  table.table3 th {background-color:#00A4CC10; border: 2px solid white; transition: ease .2s; }""")
	html_file.write("""table.table3 th {font-weight: normal; color:#00A4CC; }""")
	html_file.write("""table.table3 tr:hover{background-color:#00A4CC20;}""")
	
	html_file.write("""input { display: none;}""")

	html_file.write("""div.content { text-align: left; line-height: 150%; border: 2px solid #00A4CC; margin: 0px 0px 0px 0px; border-radius: 5px; overflow: auto; }""")
	html_file.write("""div.content { transition: ease .2s;}""")			

	html_file.write("""label { font-size: 80%; display: inline-block; font-weight: normal; padding: 5px 5px; margin: 3px 0px 3px 0px; cursor: pointer; background: #E0F8FD; border-radius: 10px; color: black; }""")
	html_file.write("""label {transition: ease .2s; box-shadow: 0 4px 4px -2px rgba(0,0,0,0.5); }""")
	html_file.write("""label:hover {background: #00A4CC; color: white; }""")

	html_file.write(""".link { color: #00A4CC; font-weight: bold; border-radius: 10px; padding: 5 0 5 0; }""")
	# html_file.write(""".link { transition: ease 1s; }""")			
	# html_file.write(""".link:hover {color: white; background: #00A4CC50; font-weight: bold; }""")

	html_file.write("""input + label + div.content { display: none; }""")
	html_file.write("""input:checked + label + div.content { display: block; }""")
	html_file.write("""input:checked + label {background-color: #00A4CC75;}""")
	html_file.write("""</style><body>""")

# summary

	if meaning == "":
		# html_file.write(f"""<h3>{pali}</h3>""")
		html_file.write(f"""<div class="content"><p>{pos}. <b>{buddhadatta}</b> [under construction]</p></div>""")
		text_long.write(f"""{pali}. {grammar}. {buddhadatta}. [under construction]""")

	
	if meaning != "":
		html_file.write(f"""<div class="content"><p>{pos}""")

		if case != "":
			html_file.write(f""" ({case})""")

		html_file.write(f""". <b>{meaning}.</b>""")
		if base == "":
			construction_simple = re.sub(" \[.+\] \+", "", construction)
			construction_simple = re.sub("> .+? ", "", construction_simple)
			construction_simple = re.sub("<br/>.+", "", construction_simple)
			html_file.write(f""" [{construction_simple}]""")

		if base != "":
			family_plus = re.sub(" ", " + ", family)
			construction_oneline = re.sub("<br/>.+", "", construction) 
			construction_truncated = re.sub("(.+)(\+ .{1,7}$)", "\\2", construction_oneline)
			if re.match("^na ", construction):
				construction_na = re.sub("^(na )(.+)$", "\\1 + ", construction)
			else:
				construction_na = ""
			
			construction_reconstructed = f"{construction_na}{family_plus} + {root_sign} {construction_truncated}"
			html_file.write(f""" [{construction_reconstructed}]""")
		html_file.write(f"""</p></div>""")

# grammar
		html_file.write(f"""<input type="checkbox"  id="grammar{pali_}" /><label for="grammar{pali_}">grammar</label>""")
		html_file.write(f"""<div class="content">""")

		html_file.write(f"""<table class = "table1"><tr><th>pāli</th><td>{pali2}</td></tr>""")
		html_file.write(f"""<tr><th>grammar</th><td>{grammar}""")
		text_long.write(f"{pali}. {grammar}")

		if neg != "":
			html_file.write(f""", {neg}""")
			text_long.write(f""", {neg}""")

		if verb != "":
			html_file.write(f""", {verb}""")
			text_long.write(f""", {verb}""")

		if trans != "":
			html_file.write(f""", {trans}""")
			text_long.write(f""", {trans}""")

		if case != "":
			html_file.write(f""" ({case})""")
			text_long.write(f""" ({case})""")

		html_file.write(f"""</td></tr>""")
		html_file.write(f"""<tr valign="top"><th>meaning</th><td><b>{meaning}</b>""")
		text_long.write(f""". {meaning}""")

		if lit != "":
			html_file.write(f"""; lit. {lit}""")
			text_long.write(f"""; lit. {lit}""")
		html_file.write(f"""</td></tr>""")

		if root != "":
			html_file.write(f"""<tr valign="top"><th>root</th><td>{root} {root_grp} {root_sign} ({root_meaning})</td></tr>""")
			text_long.write(f""". root: {root} {root_grp} {root_sign} ({root_meaning})""")

		if root_in_comps != "" and root_in_comps != "0":
			html_file.write(f"""<tr valign="top"><th>√ in comps</th><td>{root_in_comps}</td></tr>""")
			text_long.write(f""", {root_in_comps} in comps""")

		if base != "":
			html_file.write(f"""<tr valign="top"><th>base</th><td>{base}</td></tr>""")
			text_long.write(f""". base: {base}""")

		if construction != "":
			html_file.write(f"""<tr valign="top"><th>construction</th><td>{construction}</td></tr>""")
			construction_text = re.sub("<br/>", ", ", construction)
			text_long.write(f""". constr: {construction_text}""")

		if derivative != "":
			html_file.write(f"""<tr valign="top"><th>derivative</th><td>{derivative} ({suffix})</td></tr>""")
			text_long.write(f""". derivative: {derivative} ({suffix})""")

		if pc != "":
			html_file.write(f"""<tr valign="top"><th>phonetic</th><td>{pc}</td></tr>""")
			pc_text = re.sub("<br/>", ", ", pc)
			text_long.write(f""". phonetic: {pc_text}""")

		if comp != "":
			html_file.write(f"""<tr valign="top"><th>compound</th><td>{comp} ({comp_constr})</td></tr>""")
			text_long.write(f""". compound: {comp} ({comp_constr})""")

		if ant != "":
			html_file.write(f"""<tr valign="top"><th>antonym</th><td>{ant}</td></tr>""")
			text_long.write(f""". antonym: {ant}""")

		if syn != "":
			html_file.write(f"""<tr valign="top"><th>synonym</th><td>{syn}</td></tr>""")
			text_long.write(f""". synonym: {syn}""")

		if var != "":
			html_file.write(f"""<tr valign="top"><th>variant</th><td>{var}</td></tr>""")
			text_long.write(f"""variant: {var}""")

		if comm != "":
			html_file.write(f"""<tr valign="top"><th>commentary</th><td>{comm}</td></tr>""")
			comm_text = re.sub("<br/>", " ", comm)
			comm_text = re.sub("<b>", "", comm_text)
			comm_text = re.sub("</b>", "", comm_text)
			text_long.write(f""". commentary: {comm_text}""")

		if notes != "":
			html_file.write(f"""<tr valign="top"><th>notes</th><td>{notes}</td></tr>""")
			text_long.write(f""". notes: {notes}""")

		if cognate != "":
			html_file.write(f"""<tr valign="top"><th>cognate</th><td>{cognate}</td></tr>""")
			text_long.write(f""". eng congante: {cognate}""")

		if link != "":
			html_file.write(f"""<tr valign="top"><th>link</th><td><a href="{link}">{link}</a></td></tr>""")
			text_long.write(f""". link: {link}""")

		if non_ia != "":
			html_file.write(f"""<tr valign="top"><th>non ia</th><td>{non_ia}</td></tr>""")
			text_long.write(f""". non IA: {non_ia}""")

		if sk != "":
			html_file.write(f"""<tr valign="top"><th>sanskrit</th><td><i>{sk}</i></td></tr>""")
			text_long.write(f""". sanskrit: {sk}""")

		sk_root_mn = re.sub("'", "", sk_root_mn)
		if sk_root != "":
			html_file.write(f"""<tr valign="top"><th>sk root</th><td><i>{sk_root} {sk_root_cl} ({sk_root_mn})</i></td></tr>""")
			text_long.write(f""". sk root: {sk_root} {sk_root_cl} ({sk_root_mn})""")

		html_file.write(f"""</table>""")
		html_file.write(f"""</div>""")

# examples
					
		if eg1 != "" and eg2 != "":
			html_file.write(f"""<input type="checkbox" id="title2{pali_}" /><label for="title2{pali_}">examples</label>""")
			html_file.write(f"""<div class="content">""")
			html_file.write(f"""<p>{eg1}<br><span style="color:{colour2}"><i>{source1} {sutta1}</i></span>""")
			html_file.write(f"""<p>{eg2}<br><span style="color:{colour2}"><i>{source2} {sutta2}</i></span>""")
			html_file.write(f"""</div>""")
			
		elif eg1 != "" and eg2 == "":
			html_file.write(f"""<input type="checkbox" id="title2{pali_}" /><label for="title2{pali_}">example</label>""")
			html_file.write(f"""<div class="content">""")
			html_file.write(f"""<p>{eg1}<br><span style="color:{colour2}"><i>{source1} {sutta1}</i></span>""")
			html_file.write(f"""</div>""")
					
# inflection table

	if pos not in indeclinables:
		try:
			with open(f"/home/bhikkhu/Bodhirasa/Dropbox/dpd/inflection list generator/output/html/{pali}.html") as table_data:
				table_data_read = table_data.read()
				# table_data_read = re.sub("<table>", "", table_data_read)
				# table_data_read = re.sub("\t", "", table_data_read)
		except:
			error_log.write(f"error reading inflection table - <b>{pali}</b>.html\n")

		if pos in declensions:
			html_file.write(f"""<input type="checkbox" id="title3{pali_}" /><label for="title3{pali_}">declension</label>""")
		if pos in conjugations:
			html_file.write(f"""<input type="checkbox" id="title3{pali_}" /><label for="title3{pali_}">conjugation</label>""")

		html_file.write(f"""<div class="content">""")
		html_file.write(f"""{table_data_read}</table>""")

		if pos != "sandhi" and pos != "idiom":
			if pos in declensions:
				html_file.write(f"""<p>Spot a mistake in the declension table? Something missing? """)
				html_file.write(f"""<a class="link" href="https://docs.google.com/forms/d/e/1FAIpQLSfKUBx-icfRCWmhHqUwzX60BVQE21s_NERNfU2VvbjEfE371A/viewform?usp=pp_url&entry.1433863141=GoldenDict+{today}" target="_blank">Report it here!</a>""")

			if pos in conjugations:
				html_file.write(f"""<p>Spot a mistake in the declension table? Something missing? """)
				html_file.write(f"""<a class="link" href="https://docs.google.com/forms/d/e/1FAIpQLSdAL2PzavyrtXgGmtNrZAMyh3hV6g3fU0chxhWFxunQEZtH0g/viewform?usp=pp_url&entry.1433863141=GoldenDict+{today}" target="_blank">Report it here!</a>""")
		html_file.write(f"""</div>""")


# root family

	try:
		with open(f"/home/bhikkhu/Bodhirasa/Dropbox/dpd/root families generator/subfamily html/{root} {root_grp} {root_meaning} {family}.html") as table_data:
			table_data_read = table_data.read()
			# table_data_read = re.sub("<table>", "", table_data_read)
			# table_data_read = re.sub("\t", "", table_data_read)
	except:
			if root != "":
				error_log.write(f"""error reading subfamily - {pali} - {root} {root_grp} {root_meaning} {family}\n""")

	if family != "":
		html_file.write(f"""<input type="checkbox" id="title4{pali_}" /><label for="title4{pali_}">root family</label>""")
		html_file.write(f"""<div class="content">""")
		html_file.write(f"""<p><b>{pali_clean}</b> belongs to the root family <b>{family}</b> ({root_meaning})</p>""")
		html_file.write(f"""{table_data_read}""")
		html_file.write(f"""</div>""")

# compound families
	with open ("/home/bhikkhu/Bodhirasa/Dropbox/dpd/compound families generator/compound_family_list.csv", "r") as cfl: 
		cfl = cfl.read()
	
	if family2 != "" and meaning != "":
		html_file.write(f"""<input type="checkbox" id="title5{pali_}" /><label for="title5{pali_}">compound family</label>""")
		html_file.write(f"""<div class="content">""")
	
		compound_family_list = []
		compound_family_list = list(family2.split())

		for cf in compound_family_list:
				try:
					with open(f"/home/bhikkhu/Bodhirasa/Dropbox/dpd/compound families generator/output/{cf}.html", "r") as data:
						data_read = data.read()
						html_file.write(f"<p>compounds which contain <b>{cf}</b></p>")
						html_file.write(f"{data_read}")
						
				except:
					if family2 != "":
								error_log.write(f"""error reading compound family - {pali} - {cf}\n""")

		html_file.write(f"""</div>""")

	if family2 == "" and meaning != "" and re.search(fr"\b{pali_clean}\b", cfl):
		html_file.write(f"""<input type="checkbox" id="title5{pali_}" /><label for="title5{pali_}">compound family</label>""")
		html_file.write(f"""<div class="content">""")

		try:
			with open(f"/home/bhikkhu/Bodhirasa/Dropbox/dpd/compound families generator/output/{pali_clean}.html", "r") as data:
				data_read = data.read()
				html_file.write(f"<p>compounds which contain <b>{pali_clean}</b></p>")
				html_file.write(f"{data_read}")
				
		except:
			if family2 != "":
						error_log.write(f"""error reading compound family - {pali} - {family2}\n""")
		html_file.write(f"""</div>""")

# feedback
	html_file.write(f"""<input type="checkbox" id="title6{pali_}" /><label for="title6{pali_}">feedback</label>""")
	html_file.write(f"""<div class="content">""")
	html_file.write(f"""<p>Did you spot a mistake in the dictionary? Have a bright idea?  """)
	html_file.write(f"""<a class="link" href="https://docs.google.com/forms/d/e/1FAIpQLSf9boBe7k5tCwq7LdWgBHHGIPVc4ROO5yjVDo1X5LDAxkmGWQ/viewform?usp=pp_url&entry.1433863141=GoldenDict {today}" target="_blank">Report it here!</a>""")
	html_file.write(f"""<span> Your feedback and suggestions are most apprecaited.</span>""")

	html_file.write(f"""<p>You are using <b>DPD GoldenDict</b> generated on <b>{today}</b> Please <a class="link" href="https://drive.google.com/drive/folders/1W66OXwX6KQHnK0b-7E1enZJQLjXehit4?usp=sharing">check here</a> every full moon for an update.""")
	html_file.write(f"""<p>Email <a class="link" href = "mailto:digitalpalidictionary@gmail.com?subject=Update me!&body=Please send me DPD updates as soon as they're available.">digitalpalidictionary@gmail.com</a> to get updates directly to your inbox.</p>""")
	html_file.write(f"""</div>""")
	html_file.write(f"""</body></html>""")

	text_long.close()
	html_file.close()

# write gd.json
	
	try:
		with open(f"/home/bhikkhu/Bodhirasa/Dropbox/dpd/inflection list generator/output/synonyms/{pali}", "rb") as syn_file:
			synonyms = pickle.load(syn_file)
			synonyms = (synonyms)

	except:
		error_log.write(f"error reading synonyms - {pali}\n")
	
	headword_html =  open(f"/home/bhikkhu/Bodhirasa/Dropbox/dpd/exporter/output/html/{pali}.html", "r")
	definition_html = headword_html.read()
	headword_html.close()
	
	headword_text = open(f"/home/bhikkhu/Bodhirasa/Dropbox/dpd/exporter/output/text_long/{pali}.txt", "r")
	definition_plain = headword_text.read()
	headword_text.close()

	dict_entry = {}
	dict_entry = {"word": f"{pali}", "definition_html" : definition_html, "definition_plain": definition_plain, "synonyms" : synonyms}

	gd_js = open ("/home/bhikkhu/Bodhirasa/Dropbox/dpd/exporter/output/gd.json", "a", encoding ='utf8')
	json.dump(dict_entry, gd_js, ensure_ascii=False, indent=6)
	gd_js.close()

# ajdict

	ajdict = open ("/home/bhikkhu/Bodhirasa/Dropbox/dpd/exporter/output/ajdict.txt", "a", encoding ='utf8')
	ajdict.write(f"{definition_plain}\n")
	ajdict.close()

error_log.close()

0 akakkasa
5000 amhākañceso
10000 ummādana
15000 gulāguṇṭhikajāta
20000 dutiyaṃ
25000 paṭṭhapita 2
30000 byañjanakusala
35000 vaḍḍheti 1
40000 sannivesa 2


fix up json

In [8]:
f = open('/home/bhikkhu/Bodhirasa/Dropbox/dpd/exporter/output/gd.json', "r")
data = f.read()
data = "[" + data + "]"
data = re.sub("}{", "},{", data)
f.close()

with open("output/gd.json", "w") as json:
	json.write(data)

generate goldendict

In [9]:
from typing import List
from pathlib import Path
from simsapa.app.stardict import export_words_as_stardict_zip, ifo_from_opts, DictEntry
import json

zip_path = Path("./dpd_test.zip")

with open("output/gd.json", "r") as gd_data:
    gd_data_read = json.load(gd_data)


def item_to_word(x):
    return DictEntry(
        word = x['word'],
        definition_html = x['definition_html'],
        definition_plain = x['definition_plain'],
        synonyms = x['synonyms'],
    )

words = list(map(item_to_word, gd_data_read))

icon_path = Path("/home/bhikkhu/Bodhirasa/Dropbox/dpd/favicon/favicon_io nu circle/favicon-32x32.png")

ifo = ifo_from_opts({
    "bookname": "dpd",
    "author": "bdhrs",
    "description": "digital pāli dictionary",
    "website": "https://github.com/bdhrs",
})

export_words_as_stardict_zip(words, ifo, zip_path, icon_path)

In [10]:
os.popen(f'mv "/home/bhikkhu/Bodhirasa/Dropbox/dpd/exporter/dpd_test.zip" "dpd_test_{today}.zip"')